In [1]:
import pandas as pd
import statsmodels.api as sm

Мой предыдущий датасет (https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Titanic.csv) слишком маленький, в нём мало числовых переменных, поэтому использую датасет из второй аттестации.
Да, я делаю домашку слишком поздно. Могут встречаться методы из более поздних лекций, но постараюсь этим не злоупотреблять.

### Описание датасета

A data frame with 53940 rows and 10 variables:

price - price in US dollars ($326–$18,823)  
carat - weight of the diamond (0.2–5.01)  
cut - quality of the cut (Fair, Good, Very Good, Premium, Ideal)  
color - diamond colour, from D (best) to J (worst)  
clarity - a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))  
x - length in mm (0–10.74)  
y - width in mm (0–58.9)  
z - depth in mm (0–31.8)  
depth - total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43–79)  
table - width of top of diamond relative to widest point (43–95)

In [2]:
item = 'diamonds' # Выбрать из поля Item для вашего датасета
package = 'ggplot2' # Выбрать из поля Package для вашего датасета

df = sm.datasets.get_rdataset(item, package , cache=True).data
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### 1. Датасет широкий

### 2. Сделаем из него длинный

In [10]:
long_data_format = df.reset_index().melt(id_vars="index")
long_data_format

,index,variable,value
0,0,carat,0.23
1,1,carat,0.21
2,2,carat,0.23
3,3,carat,0.29
4,4,carat,0.31
...,...,...,...
539395,53935,z,3.5
539396,53936,z,3.61
539397,53937,z,3.56
539398,53938,z,3.74


Каждая строчка в датафрейме описывает один алмаз, а все колонки содержат его характеристики. Поэтому пришлось затянуть индекс внутрь датафрейма, чтобы идентифицировать по нему конкретный камень.

### 3. Сводная таблица

In [11]:
df.pivot_table(
    values='price',
    index='cut',
    columns='color',
    aggfunc='mean'
)

color,D,E,F,G,H,I,J
cut,,,,,,,
Fair,4291.061350,3682.312500,3827.003205,4239.254777,5135.683168,4685.445714,4975.655462
Good,3405.382175,3423.644159,3495.750275,4123.482204,4276.254986,5078.532567,4574.172638
Ideal,2629.094566,2597.550090,3374.939362,3720.706388,3889.334831,4451.970377,4918.186384
Premium,3631.292576,3538.914420,4324.890176,4500.742134,5216.706780,5946.180672,6294.591584
Very Good,3470.467284,3214.652083,3778.820240,3872.753806,4535.390351,5255.879568,5103.513274


Здесь мы видим, среднюю цену алмаза в зависимости качества и цвета. Зачем считал - не знаю. Надо же было что-то посчитать. Я ожидал, что чем выше качество и чем лучше цвет, тем выше будет и средняя цена. Но нет. Это удивило. Возможно всё дело в размере (чем выше качество - тем меньше будет размер, а он сильнее влияет на цену). Будет что поисследовать в дальнейшем.

### 4. Группировка

In [15]:
df.groupby(['color'])['carat'].agg('mean')

color
D    0.657795
E    0.657867
F    0.736538
G    0.771190
H    0.911799
I    1.026927
J    1.162137
Name: carat, dtype: float64

In [16]:
df.groupby(['cut'])['carat'].agg('mean')

cut
Fair         1.046137
Good         0.849185
Ideal        0.702837
Premium      0.891955
Very Good    0.806381
Name: carat, dtype: float64

Похоже, так и есть. Чем хуже цвет, тем больше средний размер. Алмазы самого низкого и самого высокого качества тоже выделяются средним размером, но при других значениях качества явной зависимости не видно.